In [ ]:
import os
import matplotlib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from tqdm.auto import tqdm
import pickle
import time
from nlp_utils import get_cos_sim, add_text_to_image, set_default_figure_params
from quintuplets import QuadrupletId, load_quadruplet, QUINTUPLETS_DATASET_PATH, Quintuplet, visualize_quadruplet, get_splits_ids
import seaborn as sns
# sns.set_theme()
set_default_figure_params()

In [ ]:
all_ids = sorted(os.listdir(QUINTUPLETS_DATASET_PATH))
def get_splits_ids():
    all_ids = sorted(os.listdir(QUINTUPLETS_DATASET_PATH))
    all_ids = np.random.RandomState(0).permutation(all_ids)
    return {'train': list(all_ids[100:]), 'test': list(all_ids[:100])}

In [ ]:
split_ids = get_splits_ids()

# import json
# with open('/home/dcor/roeyron/TCIE/qard_train_test_split_ids.json', 'w') as f:
#     json.dump(split_ids, f, indent=4)

In [ ]:
results_dir = '/home/dcor/roeyron/TCIE/results/qard_v3_embeddings_answer_shortly/'
# results_dir = '/home/dcor/roeyron/TCIE/results/qard_v2_embeddings/'

results_fnames = sorted(os.listdir(results_dir))
results_fpaths = [os.path.join(results_dir, fname) for fname in results_fnames]


splits_ids = get_splits_ids()
df = pd.concat([pd.read_pickle(fpath) for fpath in tqdm(results_fpaths)])
df['split'] = df.quintuplet_id.apply(lambda i: dict_id_to_split[i])
print(len(df))
df.head()

In [ ]:
def get_df_results(query_prompt, search_prompt, split, layer=None):
    assert query_prompt in ['general', 'original']
    assert search_prompt in ['general', 'original']
    i_token = -1
    layers = range(1, 33) if layer is None else [layer]
    
    qd_ids = df[['quintuplet_id', 'which']].drop_duplicates()
    qd_ids = [QuadrupletId(e.quintuplet_id, e.which) for _, e in qd_ids.iterrows()]
    qd_ids = [qd_id for qd_id in qd_ids if qd_id.quintuplet_id in splits_ids[split]]
    ddl_results = defaultdict(list)
    
    for i_layer in layers:
        for qd_id in qd_ids:
            df_qd_query = df[(df.quintuplet_id == qd_id.quintuplet_id) & (df.which == qd_id.which) & (df.question_type == query_prompt)]
            df_qd_serach = df[(df.quintuplet_id == qd_id.quintuplet_id) & (df.which == qd_id.which) & (df.question_type == search_prompt)]
            hs_query = df_qd_query[df_qd_query.image_type == 'query'].iloc[0].hidden_states[i_layer, i_token]
            hs_positive = df_qd_serach[df_qd_serach.image_type == 'positive'].iloc[0].hidden_states[i_layer, i_token]
            hs_negative = df_qd_serach[df_qd_serach.image_type == 'negative'].iloc[0].hidden_states[i_layer, i_token]
    
            sim_p = get_cos_sim(hs_query, hs_positive)
            sim_n = get_cos_sim(hs_query, hs_negative)
            correct = sim_p > sim_n
            
            ddl_results['i_layer'].append(i_layer)
            ddl_results['i_token'].append(i_token)
            ddl_results['correct'].append(correct)
            ddl_results['sim_p'].append(sim_p)
            ddl_results['sin_n'].append(sim_n)
    
    df_results = pd.DataFrame(ddl_results)
    return df_results

In [ ]:

def plot_accuracy_per_layer(df_results):
    # Assuming df_results is your DataFrame
    sr = df_results.groupby(by='i_layer')['correct'].agg(['mean', 'sem']).reset_index()
    sr = sr.rename(columns={'mean': 'accuracy', 'sem': 'error'})
    plt.figure()
    sns.set_style("whitegrid")
    ax = sns.lineplot(x='i_layer', y='accuracy', data=sr, marker='o', label='Accuracy')
    plt.fill_between(sr['i_layer'], 
                     sr['accuracy'] - sr['error'], 
                     sr['accuracy'] + sr['error'], 
                     alpha=0.2, 
                     # label='Error margin'
                    )
    best_layer = sr.loc[sr['accuracy'].idxmax(), 'i_layer']
    best_accuracy = sr['accuracy'].max()
    
    plt.scatter([best_layer], [best_accuracy], marker='*', c='green', s=70, zorder=5, label=f'Optimum')
    
    plt.xlabel('Layer')
    plt.ylabel('Accuracy')
    plt.xticks(range(0, len(sr['i_layer'])+1, 2))
    
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

# original question (use the same specific question)

In [ ]:
df_results_orig_orig_train = get_df_results('original', 'original', 'train')
best_layer = df_results_orig_orig_train.groupby('i_layer').mean().correct.argmax()
plot_accuracy_per_layer(df_results_orig_orig_train)
df_results_orig_orig_test = get_df_results('original', 'original', 'test', layer=best_layer)

test_acc = df_results_orig_orig_test[df_results_orig_orig_test.i_layer == best_layer].correct.mean()
print('test accuracy', test_acc)

# use different prompts

In [ ]:
    df_results_orig_general_train = get_df_results('original', 'general', 'train')
best_layer = df_results_orig_general_train.groupby('i_layer').mean().correct.argmax()
plot_accuracy_per_layer(df_results_orig_general_train)

df_results_orig_general_test = get_df_results('original', 'general', 'test')

test_acc = df_results_orig_general_test[df_results_orig_general_test.i_layer == best_layer].correct.mean()
print('test accuracy', test_acc)

# Visual examination of predicitons 

In [ ]:
i_token = -1

qd_ids = df[['quintuplet_id', 'which']].drop_duplicates()
qd_ids = [QuadrupletId(e.quintuplet_id, e.which) for _, e in qd_ids.iterrows()]
ddl_results = defaultdict(list)

for qd_id in qd_ids[:20]:
    df_qd = df[(df.quintuplet_id == qd_id.quintuplet_id) & (df.which == qd_id.which)]
    # for i_layer in range(0, 33):
    i_layer = best_layer
    hs_query = df_qd[df_qd.image_type == 'query'].iloc[0].hidden_states[i_layer, i_token]
    hs_positive = df_qd[df_qd.image_type == 'positive'].iloc[0].hidden_states[i_layer, i_token]
    hs_negative = df_qd[df_qd.image_type == 'negative'].iloc[0].hidden_states[i_layer, i_token]

    
    sim_p = get_cos_sim(hs_query, hs_positive)
    sim_n = get_cos_sim(hs_query, hs_negative)
    correct = sim_p > sim_n

    print(correct)
    visualize_quadruplet(QUINTUPLETS_DATASET_PATH, qd_id)
        